# Imports:

In [5]:
#!/usr/bin/python
# -*- coding: utf-8 -*-

import os
import librosa
import asyncio
import numpy as np
import pandas as pd
import scipy.stats as stats
import time
import nest_asyncio
from speechpy import feature
import aiofiles
import time
import parselmouth

# Extract frequency features:

## Get metadata:

In [21]:
project_root =  os.path.dirname(os.path.dirname(os.getcwd()))
source = os.path.join(project_root, "CommonVoice")

metadata = pd.read_csv(source + "/train.csv", delimiter='	')
data = metadata[['path','gender']]
data_male = data.loc[data['gender'] == 'male']
data_male = data_male.sample(frac=1).reset_index(drop=True)
data_female = data.loc[data['gender'] == 'female']

data = pd.concat([data_male,data_female])
data= data.sample(frac=1).reset_index(drop=True)
data['path'] = data['path'].str.replace('.mp3','.wav')

data_dict = dict(zip(data.path, data.gender))
len(data_dict)

<ipython-input-21-36542a20952e>:12: FutureWarning: The default value of regex will change from True to False in a future version.
  data['path'] = data['path'].str.replace('.mp3','.wav')


6385

## Extract features:

In [22]:
async def get_frequencies(count, file):
    file_path = os.path.join(project_root,"CommonVoice", "Full", file)

    if data_dict[file] == 'female':
        gender = 0
    if data_dict[file] == 'male': 
        gender = 1
    

    audio_data, sample_rate = librosa.load(file_path)
    
    print(f"\r{count}/{len(audio_files)}",end='')

    step = int(sample_rate/5) #3200 sampling points every 1/5 sec
    window_frequencies = []

    for i in range(0,len(audio_data),step):
        ft = np.fft.fft(audio_data[i:i+step]) #fft returns the list N complex numbers
        freqs = librosa.fft_frequencies(sr=16000, n_fft=len(ft))
        freqs = np.fft.fftfreq(len(ft)) #fftq tells you the frequencies associated with the coefficients
        imax = np.argmax(np.abs(ft))
        freq = freqs[imax]
        freq_in_hz = abs(freq *sample_rate)
        window_frequencies.append(freq_in_hz)
    return window_frequencies, gender, file

In [23]:
async def get_features(count, file):
    async with sem:
        
        frequencies, gender, file_name = await get_frequencies(count, file)

        nobs, minmax, mean, variance, skew, kurtosis =  stats.describe(frequencies)
        median   = np.median(frequencies)
        mode     = stats.mode(frequencies).mode[0]
        std      = np.std(frequencies)
        low,peak = minmax
        q75,q25  = np.percentile(frequencies, [75 ,25])
        iqr      = q75 - q25

        features_list.append([file_name, nobs, mean, skew, kurtosis, median, mode, std, low, peak, q25, q75, iqr, gender])

        # string = ','.join(str(item) for item in features_list)
        
        # async with aiofiles.open('CommonVoice_Features_test.csv', mode='a') as f:
        #     await f.write(f'\n{string}')
        

        return nobs, mean, skew, kurtosis, median, mode, std, low, peak, q25, q75, iqr

In [24]:
# #Calculo de tempo de disparo
start_time = time.time()

#inicio do Loop
loop = asyncio.get_event_loop()

#Controle de requisições por vez
sem = asyncio.Semaphore(2000)

#Array de tasks
sents = []

nest_asyncio.apply()

#Coleta as recomendações para envio
gender_list = []
file_list = []
features_list = []


keys = data_dict.keys()
project_root =  os.path.dirname(os.path.dirname(os.getcwd()))
file_path = os.path.join(project_root,"CommonVoice", "Full")
audio_files = os.listdir(file_path)
audio_files = list(filter(lambda file: file in list(keys), audio_files))

for k, file in enumerate(audio_files):
    if file.endswith('.wav'):
        sent = asyncio.ensure_future(get_features(count=k+1, file=file))
    else:
        pass

done, _ = loop.run_until_complete(asyncio.wait(sents))

Task exception was never retrieved
future: <Task finished name='Task-17' coro=<get_features() done, defined at <ipython-input-9-de0e4f15ae2e>:1> exception=TypeError("object tuple can't be used in 'await' expression")>
Traceback (most recent call last):
  File "/usr/lib/python3.8/asyncio/tasks.py", line 280, in __step
    result = coro.send(None)
  File "<ipython-input-9-de0e4f15ae2e>", line 4, in get_features
    frequencies, gender, file_name = await get_frequencies(count, file)
  File "<ipython-input-8-7c6d1dbf0c43>", line 10, in get_frequencies
    audio_data, sample_rate = await librosa.load(file_path)
TypeError: object tuple can't be used in 'await' expression
Task exception was never retrieved
future: <Task finished name='Task-21' coro=<get_features() done, defined at <ipython-input-9-de0e4f15ae2e>:1> exception=TypeError("object tuple can't be used in 'await' expression")>
Traceback (most recent call last):
  File "/usr/lib/python3.8/asyncio/tasks.py", line 280, in __step
    res

143/6385

Task exception was never retrieved
future: <Task finished name='Task-1' coro=<get_features() done, defined at <ipython-input-4-dbf118664b25>:1> exception=TypeError("object tuple can't be used in 'await' expression")>
Traceback (most recent call last):
  File "/usr/lib/python3.8/asyncio/tasks.py", line 280, in __step
    result = coro.send(None)
  File "<ipython-input-4-dbf118664b25>", line 4, in get_features
    frequencies, gender, file_name = await get_frequencies(count, file)
  File "<ipython-input-3-7c6d1dbf0c43>", line 10, in get_frequencies
    audio_data, sample_rate = await librosa.load(file_path)
TypeError: object tuple can't be used in 'await' expression
Task exception was never retrieved
future: <Task finished name='Task-10' coro=<get_features() done, defined at <ipython-input-4-dbf118664b25>:1> exception=TypeError("object tuple can't be used in 'await' expression")>
Traceback (most recent call last):
  File "/usr/lib/python3.8/asyncio/tasks.py", line 280, in __step
    resu

6385/6385

ValueError: Set of coroutines/Futures is empty.

In [25]:
dataframe_features = pd.DataFrame(features_list, columns = ['FileName', 'nobs', 'mean', 'skew', 'kurtosis', 'median', 'mode', 'std', 'low', 'peak', 'q25', 'q75', 'iqr', 'Gender'])
dataframe_features

,FileName,nobs,mean,skew,kurtosis,median,mode,std,low,peak,q25,q75,iqr,Gender
0,common_voice_pt_19564750.wav,23,362.391304,4.033082,15.434611,190.000000,0.0,817.303187,0.0,4070.0,0.000000,350.00,350.000000,1
1,common_voice_pt_19547806.wav,18,773.611111,2.409074,3.949058,230.000000,0.0,1662.819565,0.0,5575.0,0.000000,458.75,458.750000,1
2,common_voice_pt_21663026.wav,26,172.834496,0.829338,-0.656130,120.000000,90.0,111.624934,15.0,415.0,90.000000,267.50,177.500000,1
3,common_voice_pt_19435786.wav,22,783.636364,2.505184,5.398855,357.500000,0.0,1330.222283,0.0,5525.0,102.500000,498.75,396.250000,1
4,common_voice_pt_20523845.wav,17,327.461673,0.640652,-0.846446,335.000000,50.0,291.286727,45.0,890.0,55.000000,480.00,425.000000,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6380,common_voice_pt_20523676.wav,17,213.638144,1.001169,-0.439113,145.000000,45.0,195.744193,45.0,640.0,50.000000,350.00,300.000000,1
6381,common_voice_pt_20473567.wav,17,215.144976,0.909693,-0.649121,65.000000,60.0,195.288880,60.0,605.0,60.000000,325.00,265.000000,1
6382,common_voice_pt_19441121.wav,19,569.473684,2.016594,2.654423,190.000000,0.0,988.298504,0.0,3480.0,0.000000,385.00,385.000000,1
6383,common_voice_pt_19331464.wav,21,609.934479,3.106523,8.537151,238.624066,95.0,1216.859150,85.0,5335.0,110.000000,350.00,240.000000,1


In [26]:
dataframe_features.to_csv('data/CommonVoice_Features_data.csv', index=False)

# Get MFCCs

## Get Metadata

In [ ]:
project_root =  os.path.dirname(os.path.dirname(os.getcwd()))
source = os.path.join(project_root, "CommonVoice")

metadata = pd.read_csv(source + "/train.csv", delimiter='	')
data = metadata[['path','gender']]
data_male = data.loc[data['gender'] == 'male']
data_male = data_male.sample(frac=1).reset_index(drop=True)
data_female = data.loc[data['gender'] == 'female']

data = pd.concat([data_male,data_female])
data= data.sample(frac=1).reset_index(drop=True)
data['path'] = data['path'].str.replace('.mp3','.wav')

data_dict = dict(zip(data.path, data.gender))
len(data_dict)

## Extract MFCCs

In [ ]:
async def extract_MFCCs(count, file):
    async with sem:
        file_path = os.path.join(project_root,"CommonVoice", "Full", file)

        if data_dict[file] == 'female':
            gender = 0
        if data_dict[file] == 'male': 
            gender = 1
        
        audio_data, sample_rate =librosa.load(file_path)
        
        mfccs = librosa.feature.mfcc(y=audio_data, sr=sample_rate)
        
        mfccs_mean = list(np.mean(mfccs.T, axis= 0))
        
        sample_features = mfccs_mean
        sample_features.insert(0,str(file))
        sample_features.append(gender)
        
        string = ','.join(str(item) for item in sample_features)
        print(f"\r{count}/{len(audio_files)}",end='')
        async with aiofiles.open('CommonVoice_MFCCs_test.csv', mode='a') as f:
            await f.write(f'\n{string}')
        #features_list.append(sample_features)
    
        return 

In [ ]:
# #Calculo de tempo de disparo
start_time = time.time()

#inicio do Loop
loop = asyncio.get_event_loop()

#Controle de requisições por vez
sem = asyncio.Semaphore(600)

#Array de tasks
sents = []

nest_asyncio.apply()

#Coleta as recomendações para envio
gender_list = []
file_list = []
features_list = []

keys = data_dict.keys()
project_root =  os.path.dirname(os.path.dirname(os.getcwd()))
file_path = os.path.join(project_root,"CommonVoice", "Full")
audio_files = os.listdir(file_path)
audio_files = list(filter(lambda file: file in list(keys), audio_files))

for k, file in enumerate(audio_files):
    if file.endswith('.wav'):
        sent = asyncio.ensure_future(extract_MFCCs(count=k+1, file=file))
        sents.append(sent)
    else:
        pass
 
done, _ = loop.run_until_complete(asyncio.wait(sents))

# Get F0

## Extract Metadata

In [6]:
project_root =  os.path.dirname(os.path.dirname(os.getcwd()))
source = os.path.join(project_root, "CommonVoice")

metadata = pd.read_csv(source + "/train.csv", delimiter='	')
data = metadata[['path','gender']]
data_male = data.loc[data['gender'] == 'male']
data_male = data_male.sample(frac=1).reset_index(drop=True)
data_female = data.loc[data['gender'] == 'female']

data = pd.concat([data_male,data_female])
data= data.sample(frac=1).reset_index(drop=True)
data['path'] = data['path'].str.replace('.mp3','.wav')

data_dict = dict(zip(data.path, data.gender))

<ipython-input-6-e20a7cf83100>:12: FutureWarning: The default value of regex will change from True to False in a future version.
  data['path'] = data['path'].str.replace('.mp3','.wav')


In [7]:
async def extract_F0(count, file):
    async with sem:
        file_path = os.path.join(project_root,"CommonVoice", "Full", file)

        if data_dict[file] == 'female':
            gender = 0
        if data_dict[file] == 'male': 
            gender = 1

        audio_data = parselmouth.Sound(file_path)
        pitch = audio_data.to_pitch()
        pitch_values = pitch.selected_array['frequency']
        

        nobs_pitch, minmax_pitch, mean_pitch, variance_pitch, skew_pitch, kurtosis_pitch =  stats.describe(pitch_values)
        median_pitch   = np.median(pitch_values)
        mode_pitch     = stats.mode(pitch_values).mode[0]
        std_pitch      = np.std(pitch_values)
        low_pitch,peak_pitch = minmax_pitch
        q75_pitch,q25_pitch  = np.percentile(pitch_values, [75 ,25])
        iqr_pitch      = q75_pitch - q25_pitch
        
        sample_features = [nobs_pitch, mean_pitch, skew_pitch, kurtosis_pitch, median_pitch, mode_pitch, std_pitch, low_pitch, peak_pitch, q25_pitch, q75_pitch, iqr_pitch]
        sample_features.insert(0,str(file))
        sample_features.append(gender)
        
        string = ','.join(str(item) for item in sample_features)
        print(f"\r{count}/{len(audio_files)}",end='')
        async with aiofiles.open('CommonVoice_F0_test.csv', mode='a') as f:
            await f.write(f'\n{string}')
        #features_list.append(sample_features)
    
        return 

In [8]:
# #Calculo de tempo de disparo
start_time = time.time()

#inicio do Loop
loop = asyncio.get_event_loop()

#Controle de requisições por vez
sem = asyncio.Semaphore(600)

#Array de tasks
sents = []

nest_asyncio.apply()

#Coleta as recomendações para envio
gender_list = []
file_list = []
features_list = []

keys = data_dict.keys()
project_root =  os.path.dirname(os.path.dirname(os.getcwd()))
file_path = os.path.join(project_root,"CommonVoice", "Full")
audio_files = os.listdir(file_path)
audio_files = list(filter(lambda file: file in list(keys), audio_files))

for k, file in enumerate(audio_files):
    if file.endswith('.wav'):
        sent = asyncio.ensure_future(extract_F0(count=k+1, file=file))
        sents.append(sent)
    else:
        pass
 
done, _ = loop.run_until_complete(asyncio.wait(sents))

6385/6385